In [1]:
from sktime.datasets import load_from_tsfile_to_dataframe
import numpy as np
import pandas as pd

def load_from_tsfile(file_path, return_y=True):
    X, y = load_from_tsfile_to_dataframe(file_path)
    X = pd.DataFrame({i: pd.Series(x) for i, x in enumerate(X.iloc[:, 0])})
    if return_y:
        return X, y
    else:
        return X


# Test the function
file_path = './datasets/EthanolConcentration/EthanolConcentration_TRAIN.ts'
X, y = load_from_tsfile(file_path)
print(X.shape, y.shape)



C:\Users\ui572274\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 10.0.1 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


(1751, 261) (261,)


In [2]:
file_path = './datasets/EthanolConcentration/EthanolConcentration_TEST.ts'
X_test, y_test = load_from_tsfile(file_path)
print(X_test.shape, y_test.shape)

(1751, 263) (263,)


In [4]:
X_train  = X.transpose()

In [5]:
X_test = X_test.transpose()

In [6]:
#X_test = X_test.iloc[:, :26]



In [7]:
import lightgbm as lgb
import optuna
from sklearn.metrics import accuracy_score

def objective(trial):
    param = {
        'objective': 'multiclass',  # or 'multiclass' for multi-class classification
        'metric': 'multi_logloss',  # or 'multi_logloss' for multi-class classification
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    }

    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy



In [8]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



[I 2024-02-08 17:27:54,999] A new study created in memory with name: no-name-dfd147a6-e2a2-48c4-bec0-506facfb0e5c
[I 2024-02-08 17:27:57,596] Trial 0 finished with value: 0.41064638783269963 and parameters: {'num_leaves': 41, 'learning_rate': 0.02513310593440143, 'n_estimators': 252}. Best is trial 0 with value: 0.41064638783269963.
[I 2024-02-08 17:27:57,896] Trial 1 finished with value: 0.2509505703422053 and parameters: {'num_leaves': 25, 'learning_rate': 1.6991244491659455e-08, 'n_estimators': 31}. Best is trial 0 with value: 0.41064638783269963.
[I 2024-02-08 17:28:00,243] Trial 2 finished with value: 0.3726235741444867 and parameters: {'num_leaves': 177, 'learning_rate': 0.20090782802316554, 'n_estimators': 640}. Best is trial 0 with value: 0.41064638783269963.
[I 2024-02-08 17:28:02,778] Trial 3 finished with value: 0.4220532319391635 and parameters: {'num_leaves': 47, 'learning_rate': 0.00023124255531528978, 'n_estimators': 257}. Best is trial 3 with value: 0.4220532319391635.


KeyboardInterrupt: 